In [8]:
import os
import glob
import pandas as pd
import fastparquet

IDIR = 'data/clean_tick/'

In [9]:
def save_bars(df, path_to_file):
    if (os.path.exists(path_to_file)):
        fastparquet.write(path_to_file, df, append=True)
    else:
        fastparquet.write(path_to_file, df)

In [10]:
def get_time_bars(df, freq):
    time_bars = df.resample(freq, label='right').agg({'price': 'ohlc', 'volume': 'sum'})
    time_bars.columns = time_bars.columns.droplevel()
    time_bars.dropna(subset=['close'], inplace=True)
    return time_bars    

In [11]:
files = sorted(glob.glob(IDIR + 'Y*.parq'))
files

['data/clean_tick/Y2017.parq',
 'data/clean_tick/Y2018_1.parq',
 'data/clean_tick/Y2018_2.parq',
 'data/clean_tick/Y2018_3.parq',
 'data/clean_tick/Y2019_1.parq']

In [13]:
frequencies = ['1Min', '5Min', '10Min', '15Min', '30Min']
folders = [freq.lower() for freq in frequencies]

for file in files:
    df = pd.read_parquet(file)
    for freq, folder in (zip(frequencies, folders)):
        time_bars = get_time_bars(df, freq)
        save_bars(time_bars, f'data/{folder}/time.parq')